In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os



cwd = os.getcwd()

data_cwd = os.path.join(cwd, "data")

for dirname, _, filenames in os.walk(data_cwd):
    for filename in filenames:
        if "xls" in filename:
            print(os.path.join(filename))
        
## Source: https://www.morningstar.com/stocks/xnas/msft/financials


Income Statement_Annual_As_Originally_Reported_MSFT.xls
Balance Sheet_Annual_As_Originally_Reported_MSFT.xls


In [9]:
pip install xlrd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 678.5 kB/s eta 0:00:00a 0:00:01


In [34]:
df_income = pd.read_excel(os.path.join(data_cwd, "Income Statement_Annual_As_Originally_Reported_MSFT.xls"))
df_balance = pd.read_excel(os.path.join(data_cwd, "Balance Sheet_Annual_As_Originally_Reported_MSFT.xls"))


In [35]:
df_income.fillna(0, inplace=True)
df_balance.fillna(0, inplace=True)

In [36]:
df_income = df_income.replace(',','', regex=True)
df_balance = df_balance.replace(',','', regex=True)

In [37]:
df_income = df_income.astype({col: float for col in df_income.columns[1:]})

for col in df_balance.columns[1:]:
    df_balance[col] = df_balance[col].astype(float)
    

In [38]:
df_income.head()

,MSFT_income-statement_Annual_As_Originally_Reported,2018,2019,2020,2021,2022,TTM
0,Gross Profit,7.200700e+10,8.293300e+10,9.693700e+10,1.158560e+11,1.356200e+11,1.356200e+11
1,Total Revenue,1.103600e+11,1.258430e+11,1.430150e+11,1.680880e+11,1.982700e+11,1.982700e+11
2,Business Revenue,1.103600e+11,1.258430e+11,1.430150e+11,1.680880e+11,1.982700e+11,1.982700e+11
3,Cost of Revenue,-3.835300e+10,-4.291000e+10,-4.607800e+10,-5.223200e+10,-6.265000e+10,-6.265000e+10
4,Cost of Goods and Services,-3.835300e+10,-4.291000e+10,-4.607800e+10,-5.223200e+10,-6.265000e+10,-6.265000e+10


In [39]:
df_balance.head()

,MSFT_balance-sheet_Annual_As_Originally_Reported,2018,2019,2020,2021,2022
0,Total Assets,2.588480e+11,2.865560e+11,3.013110e+11,3.337790e+11,3.648400e+11
1,Total Current Assets,1.696620e+11,1.755520e+11,1.819150e+11,1.844060e+11,1.696840e+11
2,Cash Cash Equivalents and Short Term I...,1.337680e+11,1.338190e+11,1.365270e+11,1.303340e+11,1.047490e+11
3,Cash and Cash Equivalents,1.194600e+10,1.135600e+10,1.357600e+10,1.422400e+10,1.393100e+10
4,Cash,0.000000e+00,0.000000e+00,0.000000e+00,7.272000e+09,8.258000e+09


In [40]:
df_income.rename(columns={"MSFT_income-statement_Annual_As_Originally_Reported": "Name"}, inplace=True)
df_balance.rename(columns={"MSFT_balance-sheet_Annual_As_Originally_Reported": "Name"}, inplace=True)


In [41]:
df_income_T = df_income.set_index('Name').T
df_balance_T = df_balance.set_index('Name').T

In [42]:
df_income_T.columns = df_income_T.columns.str.lstrip()
df_income_T.columns = df_income_T.columns.str.rstrip()

df_balance_T.columns = df_balance_T.columns.str.lstrip()
df_balance_T.columns = df_balance_T.columns.str.rstrip()

## EPS Calculations 

In [43]:
df_results = pd.DataFrame()

In [44]:
df_results["Basic_EPS_Manual"] = df_income_T["Net Income Available to Common Stockholders"] / df_income_T["Basic Weighted Average Shares Outstanding"]
df_results["Diluted_EPS_Manual"] = df_income_T["Net Income Available to Common Stockholders"] / df_income_T["Diluted Weighted Average Shares Outstanding"]

In [45]:
df_results.head()

,Basic_EPS_Manual,Diluted_EPS_Manual
2018,inf,inf
2019,inf,inf
2020,inf,inf
2021,inf,inf
2022,9.703575,9.64695


In [46]:
df_eps_results = df_income_T.filter(regex='EPS')

In [47]:
df_eps_results

Name,Basic EPS,Basic EPS from Continuing Operations,Diluted EPS,Diluted EPS from Continuing Operations,Reported Normalized Diluted EPS,Basic EPS,Diluted EPS
2018,0.0,0.0,0.00,0.00,0.00,2.15,2.13
2019,0.0,0.0,0.00,0.00,0.00,5.11,5.06
2020,0.0,0.0,0.00,0.00,0.00,5.82,5.76
2021,0.0,0.0,0.00,0.00,0.00,8.12,8.05
2022,9.7,9.7,9.65,9.65,9.21,9.70,9.65
TTM,9.7,9.7,9.65,9.65,9.21,9.70,9.65


In [48]:
df_results.join(df_eps_results)

,Basic_EPS_Manual,Diluted_EPS_Manual,Basic EPS,Basic EPS from Continuing Operations,Diluted EPS,Diluted EPS from Continuing Operations,Reported Normalized Diluted EPS,Basic EPS,Diluted EPS
2018,inf,inf,0.0,0.0,0.00,0.00,0.00,2.15,2.13
2019,inf,inf,0.0,0.0,0.00,0.00,0.00,5.11,5.06
2020,inf,inf,0.0,0.0,0.00,0.00,0.00,5.82,5.76
2021,inf,inf,0.0,0.0,0.00,0.00,0.00,8.12,8.05
2022,9.703575,9.64695,9.7,9.7,9.65,9.65,9.21,9.70,9.65
TTM,9.703575,9.64695,9.7,9.7,9.65,9.65,9.21,9.70,9.65


## Return on Equity (ROE)

In [58]:
df_results = pd.DataFrame()

In [50]:
df_net_income = df_income_T.filter(regex='Net Income Available to Common Stockholders')
df_net_income.head()

Name,Net Income Available to Common Stockholders,Diluted Net Income Available to Common Stockholders
2018,1.657100e+10,1.657100e+10
2019,3.924000e+10,3.924000e+10
2020,4.428100e+10,4.428100e+10
2021,6.127100e+10,6.127100e+10
2022,7.273800e+10,7.273800e+10


In [55]:
df_balance_T.filter(regex='Total Assets')


Name,Total Assets
2018,2.588480e+11
2019,2.865560e+11
2020,3.013110e+11
2021,3.337790e+11
2022,3.648400e+11


In [60]:
df_results['Total Assets'] = df_balance_T['Total Assets']
df_results['Total Liabilities'] = df_balance_T.filter(regex='Total Liabilities')


In [61]:
df_results = df_results.join(df_net_income) 

In [62]:
df_results.head()

,Total Assets,Total Liabilities,Net Income Available to Common Stockholders,Diluted Net Income Available to Common Stockholders
2018,2.588480e+11,1.761300e+11,1.657100e+10,1.657100e+10
2019,2.865560e+11,1.842260e+11,3.924000e+10,3.924000e+10
2020,3.013110e+11,1.830070e+11,4.428100e+10,4.428100e+10
2021,3.337790e+11,1.917910e+11,6.127100e+10,6.127100e+10
2022,3.648400e+11,1.982980e+11,7.273800e+10,7.273800e+10


In [63]:
df_results["Total Shareholder Equity"] = df_results["Total Assets"] - df_results["Total Liabilities"]

In [64]:
df_results

,Total Assets,Total Liabilities,Net Income Available to Common Stockholders,Diluted Net Income Available to Common Stockholders,Total Shareholder Equity
2018,2.588480e+11,1.761300e+11,1.657100e+10,1.657100e+10,8.271800e+10
2019,2.865560e+11,1.842260e+11,3.924000e+10,3.924000e+10,1.023300e+11
2020,3.013110e+11,1.830070e+11,4.428100e+10,4.428100e+10,1.183040e+11
2021,3.337790e+11,1.917910e+11,6.127100e+10,6.127100e+10,1.419880e+11
2022,3.648400e+11,1.982980e+11,7.273800e+10,7.273800e+10,1.665420e+11


In [65]:
df_results["ROE %"] = (df_results["Net Income Available to Common Stockholders"] / df_results["Total Shareholder Equity"]) * 100
df_results["Diluted ROE %"] = (df_results["Diluted Net Income Available to Common Stockholders"] / df_results["Total Shareholder Equity"]) * 100

In [66]:
df_results

,Total Assets,Total Liabilities,Net Income Available to Common Stockholders,Diluted Net Income Available to Common Stockholders,Total Shareholder Equity,ROE %,Diluted ROE %
2018,2.588480e+11,1.761300e+11,1.657100e+10,1.657100e+10,8.271800e+10,20.033125,20.033125
2019,2.865560e+11,1.842260e+11,3.924000e+10,3.924000e+10,1.023300e+11,38.346526,38.346526
2020,3.013110e+11,1.830070e+11,4.428100e+10,4.428100e+10,1.183040e+11,37.429842,37.429842
2021,3.337790e+11,1.917910e+11,6.127100e+10,6.127100e+10,1.419880e+11,43.152238,43.152238
2022,3.648400e+11,1.982980e+11,7.273800e+10,7.273800e+10,1.665420e+11,43.675469,43.675469


### Debt to Equity (D/E)

In [67]:
df_results = pd.DataFrame()

In [69]:
df_results['Total Assets'] = df_balance_T['Total Assets']
df_results['Total Liabilities'] = df_balance_T.filter(regex='Total Liabilities')

df_results["Total Shareholder Equity"] = df_results["Total Assets"] - df_results["Total Liabilities"]

In [70]:
df_results["Debt_to_Equity"] = df_results['Total Liabilities'] / df_results["Total Shareholder Equity"]

In [71]:
df_results

,Total Assets,Total Liabilities,Total Shareholder Equity,Debt_to_Equity
2018,2.588480e+11,1.761300e+11,8.271800e+10,2.129283
2019,2.865560e+11,1.842260e+11,1.023300e+11,1.800313
2020,3.013110e+11,1.830070e+11,1.183040e+11,1.546921
2021,3.337790e+11,1.917910e+11,1.419880e+11,1.350755
2022,3.648400e+11,1.982980e+11,1.665420e+11,1.190679


### Quick Ratio

In [72]:
df_results = pd.DataFrame()

In [73]:
df_results['Total Assets'] = df_balance_T['Total Assets']
df_results['Total Liabilities'] = df_balance_T.filter(regex='Total Liabilities')

In [74]:
df_results['Quick Ratio'] = df_results['Total Assets'] / df_results['Total Liabilities']

In [75]:
df_results

,Total Assets,Total Liabilities,Quick Ratio
2018,2.588480e+11,1.761300e+11,1.469642
2019,2.865560e+11,1.842260e+11,1.555459
2020,3.013110e+11,1.830070e+11,1.646445
2021,3.337790e+11,1.917910e+11,1.740327
2022,3.648400e+11,1.982980e+11,1.839857
